# ImbalancedLearningRegression (0.0.1): Usage
---
## SMOGNBoost
Amit Shanbhoug, 8677407 \
Adapted from Nick Kunz's SMOGN package: https://github.com/nickkunz/smogn/blob/master/examples/smogn_example_1_beg.ipynb


## Installation

First, we install ImbalancedLearningRegression from the Github repository. Alternatively, we could install from the official PyPI distribution. However, the developer version is utilized here for the latest release. Works on Kernel: python 3.10.7, there may be some issues if you run with a different version, older or newer.

In [1]:
%%capture
## suppress install output

## install pypi release
# !pip install ImbalancedLearningRegression

## install developer version
# !pip install git+https://github.com/paobranco/ImbalancedLearningRegression.git

## Dependencies
Next, we load the required dependencies. Here we import `ImbalancedLearningRegression` to later apply Synthetic Minority Over-Sampling Technique for Regression with Gaussian Noise. In addition, we use `pandas` for data handling, and `seaborn` to visualize our results.

In [2]:
## load dependencies
## load libraries
import numpy as np
import pandas as pd
import sklearn
import math

from sklearn import tree
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import seaborn
import matplotlib.pyplot as plt
from ImbalancedLearningRegression.smogn import smogn
from ImbalancedLearningRegression.smogn_boost import smogn_boost
#from ImbalancedLearningRegression import *
import ImbalancedLearningRegression as iblr
from sklearn.model_selection import train_test_split

# Data Description
The dataset is about red wine, all features have numeric values (which is necessary)

## Loading the Data
Below, we load our data (Imbalanced Insurance Data set), In this case, we name our training set `data` and test data as 'test-data'. 


In [3]:
#ds = pd.read_csv("https://raw.githubusercontent.com/paobranco/ImbalancedLearningRegression/SMOGNBoost/data/housing.csv")
#ata = "https://raw.githubusercontent.com/paobranco/ImbalancedLearningRegression/SMOGNBoost/data/College.csv"
#test_data = "https://raw.githubusercontent.com/paobranco/ImbalancedLearningRegression/SMOGNBoost/data/CollegeTest.csv"
#print(len(ds))

data = pd.read_csv("https://raw.githubusercontent.com/paobranco/ImbalancedLearningRegression/SMOGNBoost/data/red_wine.csv")
test_data = pd.read_csv("https://raw.githubusercontent.com/paobranco/ImbalancedLearningRegression/SMOGNBoost/data/red_wine_test.csv")

## Introduction to SMOGNBoost
Here we cover the focus of this example. We call the `smogn_boost` function from this package (`ImbalancedLearningRegression.smogn_boost`) and satisfy the minimum required arguments: `data` and `y`.

* data: this argument takes a training data set
* test_data: this argument takes a test data set
* y: this argument takes a string, which specifies a response variable by header name 
* TotalIterations: this argument takes a positive integer, which specifies the total number of iterations
* pert: perturbation / noise percentage
* replace: sampling replacement (bool)
* k: num of neighs for over-sampling (pos int)
* error_threshold: this argument takes a positive integer, which specifies an error threshold 
* rel_thres: user defined relevance threshold 
* samp_method: "balance or extreme" - sampling method is perc

In [4]:
## conduct smogn
#smogn = smogn(data, y= "SalePrice")

In [5]:
## conduct smogn_boost
smogn_boost = smogn_boost(data, test_data, y="volatile acidity", TotalIterations=2)

Dt Test Predictions:  [1. 1.]
Weights:  0.0006459948320413437
Dt Distribution:  [0.00064599 0.00064599 0.00064599 ... 0.00064599 0.00064599 0.00064599]
phi control:  {'method': 'auto', 'num_pts': 3, 'ctrl_pts': [0.12, 0, 0, 0.52, 0, 0, 1.0, 1, 0]}
Phi Control Points:  [0.12, 0, 0, 0.52, 0, 0, 1.0, 1, 0]


r_index: 100%|##########| 69/69 [00:00<00:00, 425.38it/s]


y_oversampled:  0       0.769968
1       0.769968
2       1.147500
3       0.805263
4       1.147500
          ...   
1540    0.520000
1542    0.855000
1543    0.440000
1544    0.370000
1546    0.570000
Name: volatile acidity, Length: 1407, dtype: float64
dt test predictions:  [1.         1.         1.3425     0.49       0.65       0.665
 0.80888889 0.665      0.735      0.43       0.88367978 0.735
 0.52       0.57       0.57       0.57       0.56       0.56
 0.56       0.67       0.31       0.56       0.65       0.5
 0.44       0.47       0.935      0.29       0.44       0.37
 0.29       0.36       0.48       0.64       0.4        0.64
 0.64       0.44       0.37       0.36       0.36       0.48
 0.38       0.44       0.53       0.54       0.42       0.52
 0.88486188 0.42       0.42       0.88486188 0.37      ]
Model Error:  0.6699507389162561
Dt Data Predictions:  [0.66       0.88       0.76       ... 0.36       0.57       1.00197044]
Dt Distribution:  [0.00064599 0.00064599 0.000645

r_index: 100%|##########| 69/69 [00:00<00:00, 385.84it/s]


y_oversampled:  0       0.769968
1       1.147500
2       1.147500
3       0.928226
4       1.147500
          ...   
1542    0.855000
1543    0.440000
1544    0.370000
1546    0.570000
1547    0.600000
Name: volatile acidity, Length: 1407, dtype: float64
dt test predictions:  [1.         1.         1.3425     0.49       0.65       0.665
 0.80888889 0.665      0.735      0.43       0.88367978 0.735
 0.52       0.57       0.57       0.57       0.56       0.56
 0.56       0.67       0.31       0.56       0.65       0.5
 0.44       0.47       0.935      0.29       0.44       0.37
 0.29       0.36       0.48       0.64       0.4        0.64
 0.64       0.44       0.37       0.36       0.36       0.48
 0.38       0.44       0.53       0.54       0.42       0.52
 0.88486188 0.42       0.42       0.88486188 0.37       0.37
 0.33       0.835      0.71       0.86249275 0.64       0.79
 0.53       0.54       0.79       0.38       0.69       0.69
 0.69       0.61       0.61       0.61       0.67 

## Results
After conducting SMOGNBoost, we briefly examine the results. 






## Conclusion
TO DO

## References

Branco, P., Torgo, L., Ribeiro, R. (2017). SMOGN: A Pre-Processing Approach for Imbalanced Regression. Proceedings of Machine Learning Research, 74:36-50. http://proceedings.mlr.press/v74/branco17a/branco17a.pdf.

Torgo, L., Ribeiro, R. P., Pfahringer, B., & Branco, P. (2013, September). Smote for regression. In Portuguese conference on artificial intelligence (pp. 378-389). Springer, Berlin, Heidelberg. https://researchcommons.waikato.ac.nz/bitstream/handle/10289/8518/smoteR.pdf?sequence=23

Kunz, N. (2019). SMOGN: Synthetic Minority Over-Sampling for Regression with Gaussian Noise (Version 0.1.0). Python Package Index.
https://pypi.org/project/smogn. 

Gareth, J., Daniela, W., Trevor, H., & Robert, T. (2013). An introduction to statistical learning: with applications in R. Spinger.
http://www-bcf.usc.edu/~gareth/ISL/data.html.

